In [7]:
import os
from crewai import Agent, Task, Crew, LLM, Process
from dotenv import load_dotenv
import streamlit as st
import litellm

load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

In [8]:
token_counter = 0

def litellm_callback(model_response):
    global token_counter
    if "usage" in model_response:
        usage = model_response["usage"]
        token_counter += usage.get("total_tokens", 0)

# On attache le callback
litellm.success_callback = [litellm_callback]

In [17]:
llm_planification = LLM(
    model="groq/llama-3.1-8b-instant", 
    temperature=0, 
    api_key=api_key,
    max_retries=3,          
    timeout=60
)

solo_ai = Agent(
    role="Assistant Voyage",
    goal=f"traduction.",
    backstory="Vous êtes un assistant spécialisé dans la traduction de texte en markdown.",
    llm=llm_planification,
)
t_unique = Task(
    description="traduire le texte en markdown.",
    expected_output="traduction correcte",
    agent=solo_ai
)
crew = Crew(agents=[solo_ai], tasks=[t_unique])

In [18]:

results = crew.kickoff()
print(crew.usage_metrics)
print(results.token_usage)


total_tokens=0 prompt_tokens=0 cached_prompt_tokens=0 completion_tokens=0 successful_requests=0
total_tokens=0 prompt_tokens=0 cached_prompt_tokens=0 completion_tokens=0 successful_requests=0


In [11]:
print(f"Texte traduit : {results.raw}")
print(f"Tokens via kickoff() : {results.token_usage.total_tokens}")
print(f"Tokens via Callback LLM : {token_counter}")

Texte traduit : # Titre principal
## Sous-titre
### Sous-sous-titre

Le texte en markdown est un format de texte qui permet de créer du contenu structuré et facilement lisible. Il est souvent utilisé dans les documents, les blogs et les sites web.

### Caractéristiques clés

* **Syntaxe simple** : le texte en markdown utilise une syntaxe simple et facile à comprendre, ce qui en fait un format de texte accessible à tous.
* **Structuration** : le texte en markdown permet de structurer le contenu en utilisant des titres, des listes, des tableaux, etc.
* **Flexibilité** : le texte en markdown peut être utilisé dans de nombreux contextes, des documents de travail aux sites web.

### Avantages

* **Facile à lire** : le texte en markdown est facile à lire et à comprendre, même pour les non-spécialistes.
* **Facile à écrire** : le texte en markdown est facile à écrire et à modifier, ce qui en fait un format de texte idéal pour les documents en cours de révision.
* **Compatibilité** : le texte 

In [19]:
import os
import litellm
from crewai import Agent, Task, Crew, LLM
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

# --- LE COMPTEUR GLOBAL ---
# Cette variable va accumuler tous les tokens de chaque appel API
token_usage_reel = {
    "total": 0,
    "prompt": 0,
    "completion": 0
}

# --- LA FONCTION DE CAPTURE (CALLBACK) ---
def mon_callback_perso(model_response):
    global token_usage_reel
    # On vérifie si l'API a renvoyé le dictionnaire 'usage'
    if "usage" in model_response:
        usage = model_response["usage"]
        token_usage_reel["total"] += usage.get("total_tokens", 0)
        token_usage_reel["prompt"] += usage.get("prompt_tokens", 0)
        token_usage_reel["completion"] += usage.get("completion_tokens", 0)
        print(f"DEBUG CALLBACK : +{usage.get('total_tokens', 0)} tokens détectés.")

# On attache la fonction au succès de chaque appel LiteLLM
litellm.success_callback = [mon_callback_perso]

# --- CONFIGURATION CLASSIQUE ---
llm_planification = LLM(
    model="groq/llama-3.1-8b-instant", 
    temperature=0, 
    api_key=api_key
)

solo_ai = Agent(
    role="Assistant Traduction",
    goal="Traduire du texte.",
    backstory="Expert en markdown.",
    llm=llm_planification
)

t_unique = Task(
    description="Explique en 3 phrases pourquoi le format Markdown est utile.",
    expected_output="Explication claire en markdown",
    agent=solo_ai
)

crew = Crew(agents=[solo_ai], tasks=[t_unique])

# --- EXÉCUTION ---
print("Lancement de la tâche...")
results = crew.kickoff()

print("\n--- RÉSULTATS ---")
print(f"Contenu : {results.raw[:100]}...")
print(f"Tokens via CrewAI (kickoff) : {results.token_usage.total_tokens}")
print(f"Tokens RÉELS via Callback : {token_usage_reel['total']}")

Lancement de la tâche...

--- RÉSULTATS ---
Contenu : Le format Markdown est utile pour plusieurs raisons :
- **Simplicité** : Markdown est un format de m...
Tokens via CrewAI (kickoff) : 0
Tokens RÉELS via Callback : 0


In [ ]:
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

# Appel direct à l'API via LiteLLM
response = litellm.completion(
    model="groq/llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Bonjour, donne moi une phrase courte."}],
    api_key=api_key
)

# 2. Accéder aux valeurs individuelles
print("\n--- DÉTAILS ---")
print(f"Tokens Prompt (Entrée) : {response.usage.prompt_tokens}")
print(f"Tokens Completion (Sortie) : {response.usage.completion_tokens}")
print(f"Total des Tokens : {response.usage.total_tokens}")

--- BLOC USAGE COMPLET DE L'API ---
Usage(completion_tokens=16, prompt_tokens=43, total_tokens=59, completion_tokens_details=None, prompt_tokens_details=None, queue_time=0.087825808, prompt_time=0.002668288, completion_time=0.032727261, total_time=0.035395549)

--- DÉTAILS ---
Tokens Prompt (Entrée) : 43
Tokens Completion (Sortie) : 16
Total des Tokens : 59
